# Developments over time

(Coming next. Most frequently mentioned authors over time. Co-authorship / co-editorship numbers over time. Languages over time.)


## Distribution of publications per year 

The dataset contains 64169 publication years, with 87 different publication years recorded (some of them are errors). Here, only data starting in 1986 is shown (the bibliography was launched in 1992). 

![](/figures/pubyear_counts.png)
